In [1]:
import librosa
import soundfile
import os,glob,pickle
import numpy as np
import speech_recognition as sr
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [3]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [4]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("F:\Project\TRAINING_FILES_PATH\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [5]:
# Listning to the microphone and giving this output as an input to feature extractor
os.chdir('F:\Project\SpeechDataset')
r= sr.Recognizer()
with sr.Microphone() as source:
    print('Say Something')
    audio = r.listen(source)
    
    with open('speech.wav','wb') as f:
        f.write(audio.get_wav_data())
    

Say Something


In [6]:
file = "F:\Project\SpeechDataset\speech.wav"    
feature = extract_feature(file, mfcc=True, chroma= True, mel=True)   

In [7]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)
print(x_train)
print(x_test)
print(y_train)
print(y_test)

[[-5.76542969e+02  6.80835419e+01 -2.08600235e+01 ...  9.47501121e-06
   4.73092086e-06  2.85591705e-06]
 [-3.17272095e+02  3.99999008e+01 -2.21502590e+01 ...  1.28920691e-03
   2.73317029e-03  2.58369395e-03]
 [-4.65351440e+02  8.18976974e+01  2.71518345e+01 ...  2.70146236e-04
   1.22577505e-04  7.84473887e-05]
 ...
 [-4.51377350e+02  7.93719711e+01  2.99301548e+01 ...  2.93878227e-04
   2.63056951e-04  2.20310438e-04]
 [-5.55371155e+02  4.71378479e+01  1.10560827e+01 ...  1.61086486e-04
   1.04962528e-04  6.52812014e-05]
 [-5.04864716e+02  3.52971039e+01 -1.44038277e+01 ...  6.08151604e-04
   5.55269711e-04  4.47782280e-04]]
[[-4.29285217e+02  7.01943207e+01  1.20409155e+01 ...  7.22279679e-03
   2.22415361e-03  2.57196953e-03]
 [-3.23289978e+02  3.25324936e+01 -9.23356438e+00 ...  3.56153515e-03
   2.56175990e-03  1.92854577e-03]
 [-3.00783630e+02  4.27569160e+01 -1.87872143e+01 ...  1.50694912e-02
   2.82851215e-02  1.99986007e-02]
 ...
 [-4.67014404e+02  7.09190674e+01  1.2489247

In [8]:
#DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(2826, 942)


In [9]:
#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [10]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [11]:
#DataFlair - Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [12]:
#DataFlair - Predict for the test set
y_pred=model.predict(x_test)
y_pre=model.predict([feature])
print(y_pre)

['disgust']


In [13]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 88.96%
